In [4]:
#Imports
import pandas as pd

Reads the excel into dataframe

In [ ]:
df_assessments = pd.read_excel("./data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx")
df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])

Select the Country and Assessment date columns

In [ ]:
selected_row = (
    (df_assessments["Country"] == country) &
    (df_assessments['Assessment date'].dt.year == assessment_year)
)

Example using United Kingdom in 2023

In [24]:
country = 'United Kingdom'
assessment_year = 2023

# This returns a pd.Series of Trues and Falses
selected_country = df_assessments["Country"] == country
selected_year    = df_assessments['Assessment date'].dt.year == assessment_year


df_assessments[selected_country & selected_year]

,Id,Assessment date,Publication date,Country Id,Country,area EP.1,indicator EP.1.a,indicator EP.1.b,indicator EP.1.c,area EP.2,...,year metric CP.4.d.i,year metric CP.4.e.i,year metric CP.6.a.i,year metric CF.1.a.i,year metric CF.1.b.i,year metric CF.4.i,year metric CF.4.ii,year metric CF.4.iii,year metric CF.4.iv,Notes
22,86,2023-10-31,2023-01-12,22,United Kingdom,Partial,Yes,No,No,Partial,...,2021.0,2022.0,2021,2020.0,2021.0,2023,2023,2023,2023,Assessment results are based on in-depth polic...


Sum of the previous steps and creates a funciton which retrieves the relevant data based on the country and assessment year inputs

In [25]:
def get_country_data(country: str, assessment_year: int):

    selected_row = (
        (df_assessments["Country"] == country) &
        (df_assessments['Assessment date'].dt.year == assessment_year)
    )

    # Filter the data
    data = df_assessments[selected_row]

    # Selected and filter columns
    area_columns = [col for col in df_assessments.columns if col.startswith("area")]
    data = data[area_columns]
    
    # JSON does not allow for NaN or NULL. 
    # The equivalent is just to leave an empty string instead
    data = data.fillna('')

    #Rename columns
    data['country'] = country
    data['assessment_year'] = assessment_year

    remap_area_column_names = {
        col: col.replace('area ', '')
        for col in area_columns
    }

    data = data.rename(columns=remap_area_column_names)

    # Grab just the first element (there should only be one anyway)
    # and return it as a dictionary
    return data.iloc[0].to_dict()

In [26]:
get_country_data('Italy', 2024)

{'EP.1': 'Partial',
 'EP.2': 'Partial',
 'EP.3': 'Partial',
 'CP.1': 'No',
 'CP.2': 'Partial',
 'CP.3': 'Partial',
 'CP.4': 'Partial',
 'CP.5': 'Partial',
 'CP.6': 'Partial',
 'CF.1': 'No',
 'CF.2': 'Exempt',
 'CF.3': 'Partial',
 'CF.4': '',
 'country': 'Italy',
 'assessment_year': 2024}

# End of Week 2 Lab

This is the answer to the Week 2 lab action.

In [10]:
from pydantic import BaseModel

class CountryData(BaseModel):
    country: str
    assesment_year: int

In [ ]:

output = CountryData(country="United Kingdom", assesment_year=2024)

output

In [ ]:
output_dict = {"country": "United Kingdom", "assesment_year":2024}

output_dict

In [ ]:
CountryData(**output_dict)


In [14]:
# This is the data model for Metrics
class Metric(BaseModel):
    name: str
    value: str

In [ ]:
# Filter for just the metrics columns
selected_columns = [col for col in df_assessments.columns if col.startswith('metric')]
selected_columns

In [ ]:
# Get a random data point
data = df_assessments.iloc[0]

data[selected_columns]

In [ ]:
Metric(name="metric EP.2.a.i", value="-25%")

In [ ]:
data_as_dict = data[selected_columns].to_dict()

list_metrics = []
for name, value in data_as_dict.items():
    individual_metric = Metric(name=name, value=value)
    list_metrics.append(individual_metric)

list_metrics

In [19]:
def get_country_metrics(country: str, assessment_year: int):

    selected_row = (
        (df_assessments["Country"] == country) &
        (df_assessments['Assessment date'].dt.year == assessment_year)
    )

    # Filter the data
    data = df_assessments[selected_row]

    if data.empty:
        raise HTTPException(status_code=404, 
                            detail=f"There is no data for country: {country} and year: {assessment_year}")

    # Select just the metrics
    metric_columns = [col for col in df_assessments.columns 
                      if col.startswith('metric')]
    data = data[metric_columns]

    # JSON does not allow for NaN or NULL. 
    # The equivalent is just to leave an empty string instead
    data = data.fillna('')

    remap_area_column_names = {
        col: col.replace('metric ', '')
        for col in metric_columns
    }

    data = data.rename(columns=remap_area_column_names)

    data_as_dict = data.iloc[0].to_dict()

    list_metrics = []
    for name, value in data_as_dict.items():
        individual_metric = Metric(name=name, value=value)
        list_metrics.append(individual_metric)
    # Grab just the first element (there should only be one anyway)
    # and return it as a dictionary
    return list_metrics

In [ ]:
get_country_metrics('Italy', 2024)

In [ ]:
indicator_columns = [col for col in df_assessments.columns in col.startswith('indicator ')]

for indicator in indicator_columns:
    # TODO: find the metric associated with this specific indicator
    metric = [col for col in df_assessments.columns if ....]